In [4]:
import sys
import os
BASE_PATH = os.path.abspath('../..')
sys.path.append(BASE_PATH)

import logging
from work.auxiliary.logger_settings import configure_logger
from keras.callbacks import ReduceLROnPlateau
from work.auxiliary import data_functions

from work.unet import unet_model_functions 

In [5]:
LOG_PATH = os.path.join(BASE_PATH,'logs')
DATA_PATH = os.path.join(BASE_PATH,'data') # create a data folder at the save root folder with the "work" folder

log_path = data_functions.create_path(LOG_PATH, 'unet_logs')

configure_logger(name="cherry_stem",
                 console_level='INFO',
                 file_level='INFO',
                 out_path=log_path)

logger = logging.getLogger(__name__)

## Define paths

In [ ]:
train_path = os.path.join(BASE_PATH,r'data\raw_data\with_maskes') #path where the "label" and "image" folder resides
dest_path =  os.path.join(BASE_PATH,r'data\unet_data\training') # the path to save the current session results
test_path = os.path.join(BASE_PATH,r'data\raw_data\images_orig') # path containig images to segment

pretrained_weights = os.path.join(DATA_PATH, r'unet_data\training\2019-09-30_07-19-46\unet_cherry_stem.hdf5')

src_path = os.path.join(BASE_PATH,r'data\unet_data\training\2019-10-15_16-21-49') # optional path to load a pretrained model
                                                                                  # to continue trainig
steps=None # number of steps to select where to continue the trainig from

## Model parameters example

In [ ]:
params_dict = dict(

    train_path=train_path,
    save_path=dest_path,
    x_folder_name='image', # the name of the 'image' folder within the train path (contains the X data or images)
    y_folder_name='label', # the name of the 'label' folder within the train path (contains the Y data or labels)

    save_name='cherry', # name used as basename for all output which is save (the base file name)

    pretrained_weights=pretrained_weights, # path to pretrained wights which can be loaded for this training session
    checkpoint=None, #optional, path to pretrained folder of pretrained model which can be loaded for this training session
    data_gen_args=dict(rotation_range=180,
                       brightness_range=[0.2, 1.],
                       shear_range=5,
                       zoom_range=0.5,
                       horizontal_flip=True,
                       vertical_flip=True,
                       fill_mode='nearest'),# augmentaion arguments for the imageDataGen module in keras
    seed=78,# Random seed used in the generation of the train images

    optimizer='Adam',# name of the optimizer to use, can be any of the optimizer in: https://keras.io/optimizers/
    optimizer_params=dict(lr=1e-5,
                          amsgrad=False),

    loss='binary_crossentropy',
    metrics=[],# list of metrics which will be monitored during trainig

    target_size=(256, 256), # the source images are resized for computetional efficiency to this target size
    color_mode='grayscale', # where to train of rgb images or convert to grayscale

    batch_size=10,  # my GPU cant handel any more

    epochs=15,
    steps_per_epoch=100,
    validation_split=0.2,
    validation_steps=200,

     tensorboard_update_freq=250, #update frequency for the metrics and iamges displayed in tensorboard
    weights_update_freq='epoch', #frequency to save the model to chekpoint, if 'epoch' than after each trainig epoch
    save_weights_only=True, # whether to save just the model weights or the model itself
    ontop_display_threshold=0.4, # creates binary masks for model predicionts with pixel values greater than this, which are
    #shown in tensorboard
    callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2,
                                 verbose=1, mode='auto', min_delta=0.0001,
                                 cooldown=0, min_lr=1e-6)])


In [ ]:
#load exsisting model

update_dict = dict(tensorboard_update_freq=5000,
                   weights_update_freq=20000,
                   optimizer_params=dict(lr=1e-6)) # optional dictionay to update the loaded model parameters 


model = unet_model_functions.ClarifruitUnet.load_model(src_path,steps)
model.update_model(**update_dict)

In [ ]:
keras_logs_path=model.set_model_for_train()
print(f"tensorboard --logdir={keras_logs_path}") #commad for activating tensorboard for this session

In [ ]:
model.fit_unet()

In [ ]:
model.prediction(test_path,dest_path)
